# Juliaで数値計算の間違い集

この文章は筑波大学大学院システム情報工学研究科リスク工学専攻の紀要[「リスク工学研究」Vol. 16, pp.27-30](https://www.risk.tsukuba.ac.jp/pdf/bulletin16.pdf)に掲載された内容をJuliaを用いた計算例を加えて加筆修正したものです。


## はじめに

「数値計算・数値解析」という言葉、日頃よく耳にすると思いますが、**「数値計算・数値解析」とは何でしょう**か。本稿は、まずこの二つの言葉を明確に区別する事から始めます。今日、筆者が知る限り、「数値解析」は同じ言葉で二つの使い方をされます。一つは、「数値解析」とは数値を用いて代数的操作によって解くことができない数学の問題を解決する手法とする使い方、数値計算・数値実験・数値シミュレーションなどと同義のように使用されます。もう一つは、「数値解析」とは応用数学の一分野で、上記の数学問題を、数値を用いて近似的に解く手法に関する数学的な概念を研究する分野とする使い方。筆者は後者の立場で、数値解析という言葉を使います。すなわち、**数値を用いた計算を数値計算、数値計算に関する数学分野を数値解析**とします。

さて、区別をはっきりさせたところで質問があります。数値計算の結果はいつも正しいのでしょうか？普段、数値計算を頻繁に利用するけれども、その計算結果を闇雲に信じてはいませんか。あるいは「数値計算には誤差がある」という知識がある人は計算結果は正しいかもしれないし、間違っているときもあるだろうと認識されているかもしれません。本稿の目的は、数値計算の結果が間違える事がある例をいくつか紹介し、数値計算の不正確さ・不確実性を明確にし、数値計算による間違いのリスクを明らかにする事です。

では、古典的な電卓（電子卓上計算機）の計算結果から紹介します。お手元の電卓 に以下の計算をさせてみて下さい。

$$
\begin{array}{rrrl}
&100 & 000 & 000\\
-)& 9 & 999  & 999.99  \\
\hline
\end{array}
$$

答えは

$$
\begin{array}{rrrl}
&100 & 000 & 000\\
-)& 9 & 999  & 999.99  \\
\hline
&\color{red}{90} &\color{red} {000} &\color{red} {000.0}~(!?)
\end{array}
$$

流石にこれはまずい。一目で間違えに気づきます。筆者の手元のiPhoneの電卓は9桁の表示領域があり、これに2桁加えた11桁の固定小数で計算が実行されます。すると

$$
\begin{array}{rrrl}
&100 & 000 & 000.\color{red} {00}|\\
-)& 9 & 999  & 999.99|\color{red} {00}\\
\hline
&90 & 000 & 000.01
\end{array}
$$

一見、正しい答えが計算されているように見えますが、表示できる数値が9桁のため

$$
\begin{array}{rrrl}
&100 & 000 & 000.00\\
-)& 9 & 999  & 999.99{00}\\
\hline
&90 & 000 & 000.0\not{1}
\end{array}
$$

このように最後の「１」は消えてしまいます。数値計算が間違えた時、一番の問題は結果が間違っていても実行した計算機は知らせてくれない事です。この問題は結果を見れば間違いに気づきますが、少し複雑化した次の場合はどうでしょうか。

## フェルマーの最終定理

> $3$ 以上の自然数 $n$ について、
> 
> $$
 	x^n + y^n = z^n
$$
> 
> となる $0$ でない自然数 $(x, y, z)$ の組が存在しない。

これは17世紀、数学者のフェルマーによって古代ギリシャの数学者ディオファントスの著作「算術」の余白に「この定理に関して、私は真に驚くべき証明を見つけたが、この余白はそれを書くには狭すぎる」というメモとともに書かれた定理で、長い間数学の未解決問題として残っていた問題です。そして1995年、この定理はイギリスの数学者ワイルズによって完全に証明され （文献1）、その証明の顛末を含めて、大変脚光を浴びました。

筆者はこの定理の証明に関して、これ以上書ける事はないのですが、数値計算を使うと以下の主張ができます。


> $(x,y,z)=(139,954,2115)$ において
> 
> $$
 	139^3 + 954^3 = 2115^3
 $$
> 
> が成り立つ！？

これはフェルマーの最終定理の反例です。根拠として以下を示します。

In [4]:
x = Int32(139); y = Int32(954); z = Int32(2115);
if x ^ 3 + y ^ 3 == z ^ 3
    println("Counter example of Fermat's theorem")
    println("(x, y, z) = ($x, $y, $z)")
end

Counter example of Fermat's theorem
(x, y, z) = (139, 954, 2115)


もちろん（！）筆者の数値計算結果は間違っており、これは符号付き整数の計算機内での表現が引き起こす間違いが原因です。正しくは

$$
(x^3,y^3,z^3)=(2685619,868250664,9460870875)
$$

となり反例にはなりません。**数の表現範囲（精度）が足りなかった**のです。


## Rumpの例題

では十分精度があれば、正しい結果が得られるのでしょうか？先の例では32ビット符号付き整数という数値を用いて失敗しました。もっと数の表現範囲が増える64ビット浮動小数点数（double型と呼ばれる計算機上での小数の標準的表現方法）や128ビットのdouble-double (dd)型、あるいはそれ以上の10進100桁の精度を使って計算したら…

次の例は Rumpの例題（文献2）として有名な例題です。

> $2$ 変数 $a$, $b$ を引数に持つ非線形関数
> 
> $$
	f(a,b)=(333.75-a^2)b^6+a^2(11a^2b^2-121b^4-2)+5.5b^8+\frac{a}{2b}
$$
> 
> について $a=77617$, $b=33096$ のときの値を求めよ。

この関数、最大8次の多項式があるので、手計算はまずやりたくありません。そこで数値計算の出番ですが、先ほど懲りましたので、今回はいくつかの精度（double型、dd型、100桁小数）で計算をしてみます。

注意：Juliaでdd型を実装するパッケージとしてここでは`DoubleFloats`を使う。`setprecision(BigFloat, 128)`としても同じ計算ができる。

In [14]:
using DoubleFloats
function f(a::T, b::T) where T
    return (convert(T,333.75) - a ^ 2) * b ^ 6 + a ^ 2 * (convert(T,11) * a ^ 2 * b ^ 2 - convert(T,121) * b ^ 4 - convert(T,2)) + convert(T,5.5) * b ^ 8 + a / (convert(T,2)*b)
#     return (convert(Float64,333.75) - a ^ 2) * b ^ 6 + a ^ 2 * (convert(Float64,11) * a ^ 2 * b ^ 2 - convert(Float64,121) * b ^ 4 - convert(Float64,2)) + convert(Float64,5.5) * b ^ 8 + a / (convert(Float64,2)*b)
end
# f(a, b) = 333.75b^6 + a ^ 2 * (11a ^ 2 * b ^ 2 - b ^ 6 - 121b ^ 4 - 2) + 5.5b ^ 8 + a / (2b)

a = Float32(77617)
b = Float32(33096)
c = f(a,b)
println("single: $c")

a = Float64(77617)
b = Float64(33096)
c = f(a,b)
println("double: $c")

a = Double64(77617)
b = Double64(33096)
print("dd:     ")
showall(f(a,b))
println("")

digits = 37
setprecision(Int(ceil(digits/log10(2))))
a = BigFloat(77617)
b = BigFloat(33096)
c = f(a,b)
println("mpfr:   $c")

single: 1.172604
double: 1.1726039400531787
dd:     1.1726039400531786318588349045201801
mpfr:   -0.827396059946821368141165095479816292005



### 参考文献

1. A. Wiles, Modular elliptic curves and Fermat's Last Theorem, Annals of Mathematics, 141 (3), pp. 443-551, 1995.
1. E. Loh, G. W. Walster, Rump's Example Revisited, Reliable Computing, 8 (3), pp. 245-248, 2002.
1.  https://twitter.com/mkashi/status/1176148600373633024?s=20
1. U. W. Kulisch, W. L. Miranker, The Arithmetic of the Digital Computer: A New Approach, SIAM Review, 28, pp. 1-40, 1986.
1. W. M. Kahan, The Regrettable Failure of Automated Error Analysis, A Mini-Course prepared for the conference at MIT on Computers and Mathematics, 1989.
